In [1]:
!pip3 install bs4
!pip3 install selenium
!pip3 install webdriver_manager

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [47]:
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
import urllib

import requests

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import requests
import json

In [52]:

class AmazonCrawler:

    def __init__(self):
        self.__base_url = "https://www.amazon.sg"  # Amazon SG
        # init chrome driver
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        service = Service(executable_path=ChromeDriverManager().install())
        self.driver = webdriver.Chrome(options=options, service=service)
    
    def get_product_list(self, search_term: str, limit: int = 100, amazon_category_id: str = None):
            """
            Get product urls and store in list
            """
            search_term_encoded = urllib.parse.quote_plus(search_term)

            url_list = []
            length = 0
            page = 1

            while True:
                
                if amazon_category_id:
                    url = self.__base_url + f"/s?k={search_term_encoded}&page={page}&rh=n%3A{amazon_category_id}&dc"
                else:
                    url = self.__base_url + f"/s?k={search_term_encoded}&page={page}"
                bs = self.__get_html_content(url=url)
                s = bs.find_all(attrs={"data-component-type": "s-search-result"})
                print(f"Processing url list, page: {page}")
                print(url)
                for i in s:
                    if length >= limit:
                        break
                    link_element = i.select(".a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal")
                    if link_element:
                        url_list.append(link_element[0]["href"])
                        length += 1

                if length < limit:
                    page += 1
                else:
                    break

            return url_list
    
    def get_product_details(self, product_url: str):
        data = []
        response = requests.get(product_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            product_details_section = soup.find('div', {'id': 'productDetails_feature_div'})
            print("product", product_details_section)
            if product_details_section != None:
                for detail in product_details_section.find_all('li'):
                # Extract the label of the detail (e.g. "Brand", "Manufacturer")
                    label = detail.b.text.strip()
                # Extract the value of the detail (e.g. "Apple", "Samsung")
                    value = detail.text.split(':')[1].strip()
                # Print the extracted information
                    print(f'{label}: {value}')
        return data
    
    def get_reviews_details(self, review_url, page_num):
        print('URL For reviews:', review_url)
        # Make a GET request to the product review page
        for page in range(1, page_num+1):
            review_url = review_url + f'pageNumber={page}/ref=cm_cr_arp_d_paging_btm_next_{page}?pageNumber={page}'
            response = requests.get(review_url)
            reviews = []
            if response.status_code == 200:
                # Parse the HTML content of the response using BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')
                # Find all the review containers on the page
                review_containers = soup.find_all('div', {'data-hook': 'review'})
                for container in review_containers:
                    review = {}
                    # Extract the rating of the review
                    rating_element = container.find('i', {'data-hook': 'review-star-rating'})
                    if rating_element != None:
                        rating = float(rating_element.span.text.split()[0])
                        review['rating'] = rating
                    else:
                        review['rating'] = 0.0

                    # Extract the title of the review
                    title_element = container.find('a', {'data-hook': 'review-title'})
                    if title_element != None:
                        title = title_element.text.strip()
                        review['title'] = title
                    else:
                        review['title'] = ''

                    # Extract the content of the review
                    content_element = container.find('span', {'data-hook': 'review-body'})

                    if content_element != None:
                        content = content_element.text.strip()
                        review['content'] = content
                    else:
                        review['content'] = ''

                    reviews.append(review)
                    # Print the extracted information
                    # print(f'Rating: {review.rating}')
                    # print(f'Title: {review.title}')
                    # print(f'Content: {review.content}')
                    print('Review: ', review)
        
        return reviews
    
    def get_product_ASIN(self, url: str):
        url_parts = url.split("/")
        # Get the index of "dp" in the URL parts
        dp_index = url_parts.index("dp")
        # Get the ASIN from the URL part after "dp"
        asin = url_parts[dp_index+1]
        print("Product Code", asin)
        return asin

    def crawl_content_into_csv(self, search_term: str, limit: int, amazon_category_id=None):
        product_list = self.get_product_list(
            search_term=search_term,
            limit=limit,
            amazon_category_id=amazon_category_id
        )
        with open(f'products_{search_term}.csv', 'a+', newline='', encoding="utf-8") as f:
            writer = csv.writer(f)
            for product_slug in product_list:
                row = []
                asin = self.get_product_ASIN(product_slug)
                review_url = f'https://www.amazon.sg/product-reviews/{asin}/'
                reviews = self.get_reviews_details(review_url, 6)
                print("All Reviews: ", reviews)
                product_detail_section_url = f'https://www.amazon.sg/dp/{asin}'
                # product_details = self.get_product_details(product_detail_section_url)
                # for review in reviews:
                #     row.append(review)
                print("writing new row...")
                writer.writerow(row)

    def __get_html_content(self, url: str) -> BeautifulSoup:
        self.driver.get(url)
        html = self.driver.page_source
        bs = BeautifulSoup(html, 'html.parser')
        return bs

In [53]:
ac = AmazonCrawler()

In [54]:
categories = {
    #"Women’s Fashion": "6833514051",
    #"Kitchen & Dining": "6537675051",
    #"Sports Apparel & Equipment": "6314940051",
    "DIY & Tools": "6314687051",
    #"Office Products": "6314868051"
}

In [55]:
for search_term, category_id in categories.items():
    ac.crawl_content_into_csv(search_term=search_term, limit=25, amazon_category_id=category_id)

Processing url list, page: 1
https://www.amazon.sg/s?k=DIY+%26+Tools&page=1&rh=n%3A6314687051&dc
Processing url list, page: 2
https://www.amazon.sg/s?k=DIY+%26+Tools&page=2&rh=n%3A6314687051&dc
Product Code B003ZJ8MIU
URL For reviews: https://www.amazon.sg/product-reviews/B003ZJ8MIU/
Review:  {'rating': 2.0, 'title': 'Very cheap quality items', 'content': 'Yeah very cheap below average quality tools.I suggest go with something higher in price and quality'}
Review:  {'rating': 1.0, 'title': 'Bad Quality', 'content': "Case can't open and close properly.  Tools in the box, can't hold properly.  Some pcs are no properly built."}
Review:  {'rating': 0.0, 'title': '', 'content': 'It looks really good. Not very sure about sturdiness. The comments here also make me doubtful probably somewhere between 4 and 5. But I normally just prefer positive. So I will rate 5 stars.Also Amazon makes it so chill and free worrying state of mind if you want to return it. I am just not sure about no box, no lab